In [5]:
import warnings
warnings.filterwarnings("ignore")
import torch
import sys

import torchaudio
import tgt
import os
import numpy as np
from tqdm import tqdm
import librosa
import glob

sys.path.append("../")
from pyfiles.feature_extractor import WavLMExtractor
from vocos.feature_extractors import MelSpectrogramFeatures


In [2]:
data_dir = "/mntcephfs/lab_data/shoinoue/"
dataset_dir = data_dir + "Dataset/CMU-ARCTIC/"
fs = 16000

melgen = MelSpectrogramFeatures(sample_rate=fs)
wavlm = WavLMExtractor()
speakers = [os.path.basename(a) for a in glob.glob(dataset_dir + "*")]
speakers.sort()

In [4]:
y, sr = torchaudio.load(path)

In [3]:
save = False

for spk in speakers:
    print(spk)
    wavlm_dir = f"{dataset_dir}{spk}/wavlm/"
    mel_dir = f"{dataset_dir}{spk}/mel/"
    os.makedirs(wavlm_dir, exist_ok=True)
    os.makedirs(mel_dir, exist_ok=True)
    filenames = glob.glob(dataset_dir + spk + "/wav/*")
    filenames.sort()
    for path in tqdm(filenames):
        # wavlm
        embedding = wavlm.get_feature(path).T.detach().cpu().numpy()
        savepath = wavlm_dir + os.path.basename(path)[:-4] + ".npy"
        if save:
            np.save(savepath, embedding)
            
        # mel
        y, sr = torchaudio.load(path)
        if fs!=sr:
            if y.size(0) > 1:  # mix to mono
                y = y.mean(dim=0, keepdim=True)
            y = torchaudio.functional.resample(y, orig_freq=sr, new_freq=fs)
        mel = melgen(y[0])
        savepath = mel_dir + os.path.basename(path)[:-4] + ".npy"
        if save:
            np.save(savepath, mel)

SLT


 12%|█▏        | 137/1132 [00:06<00:47, 20.92it/s]

KeyboardInterrupt



# Normalize